Users who want to use the quantum internet can buy an `endnodeHardware` unit and connect it to their quantum internet cable at home.

[May be move this paragraph somewhere else, because it talks about the layers above.]
Each `endnodeHardware` unit comes with software preinstalled that talks to rest of the layers of the quantum internet stack via the quantum internet cable. So what the users buy at the store is not just the `endnodeHardware` unit but also the software on top of it --- the sales people at the [COOL NAME FOR COMPANY THAT MAKES DEVICE] market the different iterations of this device with colorful names like [COOL NAME].

In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
%%writefile endnode_hardware.py
import sys
import math
import random
from qutip import *

sys.path.append("../..")
from _5_The_Physical_Layer.qubit_carriers.qubit import Qubit
print("imported Qubit object", Qubit)
from _5_The_Physical_Layer.qubit_carriers.photon import Photon
print("imported Photon object", Photon)

from common.global_state_container import global_state_container

class EndnodeHardware(object):
    def __init__(self, parent_endnode, qubits=1):
        print("creating endnode hardware")
#         self.id = None
        self.parent_endnode = parent_endnode
        self.global_state = global_state_container.state
        self.qubit = Qubit(self) # this qubit is used to support entanglement
        self.memory_qubit = Qubit(self) # this qubit is used to store qubits that a user needs to send
        self.fiber = None                                          
#         self.memoryQubits = []

    def connect_fiber(self, fiber):
        print("connecting fiber")
        self.fiber = fiber
        fiber.connect_node_hardware(self)

    def teleport_qubit(self): # this does the same thing as repeater.hardware.entanglement_swap.
        CNOT = cnot(N=int(math.log2(self.global_state.state.shape[0])), control=self.memory_qubit.id, target=self.qubit.id)
        new_state = CNOT * self.global_state.state * CNOT.dag()
        Z180 = rz(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.memory_qubit.id)
        Y90  = ry(90, N=int(math.log2(self.global_state.state.shape[0])), target=memory_qubit.id)
        H = Y90 * Z180
        new_state = H * new_state * H.dag()
        self.global_state.update_state(new_state)
        measurement_result1 = self.measure(self.memory_qubit[0])
        measurement_result2 = self.measure(self.qubit[0])     
        # notify the parent repeater so that it can send the classical data to
        # the other repeater.
        msg = {'msg' : "hardware: teleport done", 
               'measurement_result1' : measurement_result1,
               'measurement_result2' : measurement_result2}
        self.send_message(self.parent_endnode, msg)

    def apply_teleport_corrections(self, measurement_result1, measurement_result2):
        if measurement_result1 == 0 and measurement_result1 == 0:
            return
        elif measurement_result1 == 0 and measurement_result1 == 1:
            correction = rz(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id)
        elif measurement_result1 == 1 and measurement_result1 == 0:
            correction = rx(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id)
        elif measurement_result1 == 1 and measurement_result1 == 1:
            correction = rz(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id)
            correction = rx(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id) * correction
        new_state = correction * self.global_state.state * correction.dag()
        self.global_state.update_state(new_state)
        msg = {'msg' : "teleport corrections applied"}
        self.send_message(self.parent_endnode, msg)

    def send_message(self, obj, msg):
        obj.handle_message(msg)

    def handle_message(self, msg):
#         msg = msg.split('-')
#         # id of the sender
#         id = msg[0]
#         if msg[1] == "decohered":
#             # notify the link layer
#             msg2 = packLinkExpired(#specify which link expired#)
#             self.send_message(self.parent_endnode, msg2)
        return

    def measure(self, qubit, axis = "01"):
        print("measuring qubit in endnode hardware")
        # https://inst.eecs.berkeley.edu/~cs191/fa14/lectures/lecture10.pdf
        rho = self.global_state.state
        # construct the projectors
        P0 = tensor([identity(2) for _ in range(qubit.id)] + 
                    [basis(2,0) * basis(2,0).dag()] + 
                    [identity(2) for _ in range(qubit.id + 1, int(math.log2(self.global_state.state.shape[0])))])
        P1 = tensor([identity(2) for _ in range(qubit.id)] + 
                    [basis(2,1) * basis(2,0).dag()] + 
                    [identity(2) for _ in range(qubit.id + 1, int(math.log2(self.global_state.state.shape[0])))])
        # compute the probabilities of the 1 and 0 outcomes
        p0 = (P0 * rho).tr()
        p1 = (P1 * rho).tr() # check that p1 = 1 - p0
        # choose an outcome at random using the probabilities above.
        result = 0 if random.random() < p0 else 1
        # simulate state collapse
        new_state = P0 * rho * P0 / p0 if result == 0 else P1 * rho * P1 / p1
        # update globalState
        self.global_state.update_state(new_state)
        # return the measurement result
        return result

    def load_qubit_on_photon(self, qubit, photon):  # both qubit and photon are qubit objects
        print("loading data from local qubit onto photon")
        # swaps the state of the photon and the local qubit 
        # (the photon should be initialized to |0>. The initialization 
        # can be noisy).
        SWAP = swap(N=int(math.log2(self.global_state.state.shape[0])), targets=[qubit.id, photon.id])
        new_state = SWAP * self.global_state.state * SWAP.dag()
        self.global_state.update_state(new_state)

    def send_photon_through_fiber(self, photon, fiber):
        fiber.carry_photon(photon, self)

    def receive_photon_from_fiber(self, photon, fiber):
        print("endnode hardware receiving photon")
        # This function is called by an optical fiber to
        # alert the repeaterHardware to receive the incoming photon.
        # The repeaterHardware chooses a (physical) qubit on which to unload the 
        # qubit carried on the photon.
        self.unload_qubit_from_photon(self.qubit, photon) # confusing names.
        
    def unload_qubit_from_photon(self, qubit, photon):
        print("unloading data from photon onto local qubit")
        # swaps the state of the photon and the local qubit 
        # (the local qubit should be initialized to |0>. The initialization 
        # can be noisy). 
        SWAP = swap(N=int(math.log2(self.global_state.state.shape[0])), targets=[qubit.id, photon.id])
        new_state = SWAP * self.global_state.state * SWAP.dag()
        self.global_state.update_state(new_state)
        # notify the layers above that a qubit was received.
        if photon.header == "link":
            msg = {'msg' : "received link qubit",  # this is the standard. Document it somewhere.
                   'sender' : self.fiber.node2 if self == self.fiber.node1 else self.fiber.node1, 
                   'receiver' : self}
        else:
            msg = {'msg' : "received qubit",  # this is the standard. Document it somewhere.
                   'sender' : self.fiber.node2 if self == self.fiber.node1 else self.fiber.node1, 
                   'receiver' : self}
        if self.parent_endnode:
            self.send_message(self.parent_endnode, msg)
        photon.destroy()

    def attempt_link_creation(self, remote_node):
        # remote is a repeater object.
        # here the physical details of link creation will be implemented:
        # 1. create EPR pair. Store one half locally and load the other on a photon.
        # 2. send the photon to the remote receiver.
        self.qubit.reset()
        photon = Photon()
        Z180 = rz(180, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id)
        Y90  = ry(90, N=int(math.log2(self.global_state.state.shape[0])), target=self.qubit.id)
        H = Y90 * Z180
        new_state = H * self.global_state.state * H.dag()
        CNOT = cnot(N=int(math.log2(self.global_state.state.shape[0])), control=self.qubit.id, target=photon.id)
        new_state = CNOT * new_state * CNOT.dag()
        self.global_state.update_state(new_state)
        self.send_photon_through_fiber(photon, self.fiber)
        # 3. (for later) check somehow that we have a good link.
        # support for heralding stations and photon transmission, etc.

    def attempt_distillation(self):
        # apply gates on the qubits here
        return

imported Qubit object <class '_5_The_Physical_Layer.qubit_carriers.qubit.Qubit'>
imported Photon object <class '_5_The_Physical_Layer.qubit_carriers.photon.Photon'>
Overwriting endnode_hardware.py


# Tests

### Two EndnodeHardware objects send photons between each other

In [39]:
from qutip import *

from endnode_hardware import EndnodeHardware
from _5_The_Physical_Layer.optical_fiber.optical_fiber import OpticalFiber
from _5_The_Physical_Layer.qubit_carriers.photon import Photon

from common.global_state_container import global_state_container

global_state_container.init()

node1 = EndnodeHardware(parent_endnode=None)
node2 = EndnodeHardware(parent_endnode=None)

fiber = OpticalFiber()
node1.connect_fiber(fiber)
node2.connect_fiber(fiber)

node1.send_photon_through_fiber(Photon(), fiber)

imported Qubit object <class '_5_The_Physical_Layer.qubit_carriers.qubit.Qubit'>
imported Photon object <class '_5_The_Physical_Layer.qubit_carriers.photon.Photon'>
creating endnode hardware
creating new qubit
creating new qubit in global state
before: None
GUI not on
after: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
creating new qubit
creating new qubit in global state
before: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
GUI not on
after: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[<_5_The_Physical_Layer.qubit_carriers.qubit.Qubit object at 0x0000020E30A83188>, <_5_The_Physical_Layer.qubit_carriers.qubit.Qubit object at 0x0000020E30AABDC8>]
creating endnode hardware
creating new qubit
creating new qubit in global state
before: Quantum object: di

### Two EndnodeHardware objects create entanglement between each other

In [40]:
node1.attempt_link_creation(node2)

creating new photon
creating new qubit in global state
before: Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
GUI not on

### Unloading qubit from photon.

In [41]:
node1.unload_qubit_from_photon(node1.qubit, Photon())

creating new photon
creating new qubit in global state
before: Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[0.27596319 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.44699833 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.  

In [42]:
node1.measure(node1.qubit)

measuring qubit in endnode hardware
GUI not on


0